# Full Cooling Calculation

We are now ready to demonstrate the full cooling calculation, starting from an injection of high-energy electrons and photons, and ending in an integrated ionization and thermal history that results from this injection. 

## Initialization

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [30]:
%autoreload
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = [10,10]

import numpy as np
import pickle
import darkhistory.physics as phys
import darkhistory.spec.spectools as spectools


from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

np.set_printoptions(threshold=np.nan)

## Inputs

The inputs of the code are as follows: 

1. A `Spectrum` or `Spectra` object containing the injected electron spectrum, or the injected electron spectrum as a function of redshift, and/or 

2. A `Spectrum` or `Spectra` object containing the injected photon spectrum, or the injected photon spectrum as a function of redshift.

There are 3 modes of use. For a `Spectrum` input, the spectrum can be interpreted either as a single injection at a particular redshift, and no subsequent injections, or as a spectrum with the redshift dependence factored out, with a continuous injection at every step. For a `Spectra` input, the spectrum at each redshift step is injected. 

We will start with the single injection case here. As an example, we begin with a $\delta$-function injection of two 1 MeV electrons at some arbitrary energy, start at redshift $1+z = 2000$. Let's first initialize the spectrum. 

In [3]:
eleceng = np.logspace(1, 12.2, 100)
photeng = np.logspace(-4, 12.2, 100)
rs = 5

elec_spec_init = spectools.rebin_N_arr(np.array([1]), 
                                      np.array([1e12]),
                                      eleceng)

elec_spec_init.rs = rs

## Resolving Injected Electrons

High energy electrons ($\gg$ keV) primarily cool by inverse compton scattering (ICS), at a rate that is much faster than the smallest timestep allowable in this code ($\Delta \log (1+z) = 10^{-3}$). Lower energy electrons also cool quickly, but via atomic processes.

Low energy electrons ($< 3.5$ keV) are read out immediately and saved in a `Spectra` object. High energy electrons will be converted into a photon spectrum through ICS, and an estimate for the energy loss due to atomic cooling also produced. 

For now, we will assume only ICS for high energy electrons. 

In [4]:
raw_nonrel_ICS_tf = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ICS_nonrel.raw","rb"))
raw_rel_ICS_tf = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ICS_rel.raw","rb"))
raw_engloss_tf = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ICS_englossspec.raw","rb"))

(ics_phot_tf, ics_lowengelec_tf) = get_ics_cooling_tf(raw_nonrel_ICS_tf,
                                                     raw_rel_ICS_tf,
                                                     raw_engloss_tf,
                                                     eleceng, photeng, rs)

In [5]:
phot_spec_init = spectools.scatter(ics_phot_tf, elec_spec_init)

## Photon Cooling

Now we cool the photons that result from the high energy electron injection. Photons below 3.1 keV are considered low energy photons. For high energy photons, greater than 3.1 keV, we have transfer functions (a function of *both* injected photon energy and redshift, so they are given as a `TransFuncList`), into high energy photons, low energy photons and low energy electrons. Let's load them first. 

In [31]:
highengphot_tflist = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_photspec.raw", "rb"))
lowengphot_tflist  = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec.raw", "rb"))
lowengelec_tflist  = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec.raw", "rb"))

In [32]:
# This shouldn't be necessary. There is one negative value somewhere. Check.
for tf in highengphot_tflist:
    tf.grid_vals[np.where(tf.grid_vals < 0)] = 0
    

We now need to construct a transfer function list for *both* high- and low-energy photons. It needs to be a square matrix (so that we can take powers of the matrix for coarsening later), with dimensions given by the length of `photeng`. 

In [33]:
#Split photeng into high and low energy. 
photeng_high = photeng[photeng > 3070]
photeng_low  = photeng[photeng <= 3070]
print(highengphot_tflist[0].N_underflow)
highengphot_tflist.at_val('in_eng', photeng_high)
print(highengphot_tflist[0].N_underflow)
for tf in highengphot_tflist:
    print(tf)
    tf.rebin(photeng)

[ 6.09004918  6.08905878  6.08812727  6.0872504   6.08642597  6.08565027
  6.08492062  6.08423381  6.0835882   6.08298063  6.08240944  6.08187198
  6.08136613  6.08089076  6.08044325  6.08002251  6.07962691  6.07925435
  6.07890427  6.07857453  6.07826512  6.0779734   6.07769933  6.07744133
  6.07719884  6.0769708   6.07675615  6.07655435  6.07636433  6.07618607
  6.07601799  6.07585957  6.07571128  6.07557105  6.0754399   6.07531572
  6.07519956  6.07508984  6.07498705  6.07488964  6.07479864  6.07471247
  6.07463217  6.07455563  6.07448443  6.07441699  6.0743533   6.07429388
  6.07423767  6.07418468]
in at_val


AttributeError: 'TransferFuncList' object has no attribute 'N_underflow'

In [10]:
highengphot_tflist[0].grid_vals.shape

(54, 500)

We need to decide on the redshift step to use. The default redshift step in the transfer functions is $d \log (1+z) = 2 \times 10^{-3}$, with the transfer function evaluated at (currently) 50 redshifts, from $1+z = 30$ to $1+z=4$, so that the transfer function with $d \log(1+z) = 2 \times 10^{-3}$ can be interpolated between them. However, we can coarsen the high energy photon transfer function by some factor $f$ by multiplying the transfer function matrix by itself multiple times, producing transfer functions for a redshift step of $d \log (1+z) = f \times 2 \times 10^{-3}$. 

In [9]:
# # The redshift step to use will be coarsen_factor * 0.002
# coarsen_factor = 5
# # Generate the redshifts at which we want to get an interpolated value for the transfer function. 
# tf_rs_arr = highengphot_tflist.rs
# rs_list = np.exp(np.arange(np.log(tf_rs_arr[0]), 
#                            np.log(tf_rs_arr[-1]), 
#                            -highengphot_tflist.dlnz*coarsen_factor)
#                 )

# if coarsen_factor > 1:
#     highengphot_tflist.coarsen(coarsen_factor, delete_tfs=False)
    
# highengphot_tflist.at_val('rs',rs_list)
